In [ ]:
%pip install geopandas
%pip install matplotlib
%pip install seaborn
%pip install missingno
%pip install scipy
%pip install -e ../songdo_metr/

# Exploratory Data Analysis

In [ ]:
from datetime import datetime
from typing import Dict, List

import geopandas as gpd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
from matplotlib.ticker import FuncFormatter
from metr.components.metadata import Metadata
from metr.components.metr_imc.traffic_data import TrafficData

plt.rcParams["font.family"] = "AppleGothic"
plt.rcParams["axes.unicode_minus"] = False  # 음수 부호 깨짐 방지

## Data Preparation

In [ ]:
print("METR-LA")
roads_la: gpd.GeoDataFrame = gpd.read_file("../datasets/METRLA/miscellaneous/sensor_nodes.shp")
roads_la.explore(color="blue")

In [ ]:
print("PEMS-BAY")
roads_pems: gpd.GeoDataFrame = gpd.read_file("../datasets/PEMSBAY/miscellaneous/sensor_nodes.shp")
roads_pems.explore(color="blue")

In [ ]:
print("Incheon")
roads_imc: gpd.GeoDataFrame = gpd.read_file("../datasets/metr-imc/miscellaneous/sensor_nodes.shp")
roads_imc.explore(color="blue")

In [ ]:
df_la = pd.read_hdf("../datasets/METRLA/metr-la.h5")
df_la.iloc[:, :7]

In [ ]:
df_pems = pd.read_hdf("../datasets/PEMSBAY/pems-bay.h5")
df_pems.iloc[:, :7]

In [ ]:
metr_imc = TrafficData.import_from_hdf("../datasets/metr-imc/metr-imc.h5")
df_imc = metr_imc.data
df_imc.iloc[:, :7]

In [ ]:
metr_imc_meta = Metadata.import_from_hdf("../datasets/metr-imc/metadata.h5")
imc_metadata = metr_imc_meta.data
imc_metadata

## General Statistic Analysis

### Descriptions of Data

In [ ]:
print("Description of METR-LA")
print(df_la.reset_index(drop=True).melt()["value"].describe().apply(lambda x: format(x, '.4f')))
print("Missing values count:", df_la.isnull().sum().sum())

In [ ]:
print("Description of PEMS-BAY")
print(df_pems.reset_index(drop=True).melt()["value"].describe().apply(lambda x: format(x, '.4f')))
print("Missing values count:", df_pems.isnull().sum().sum())

기본적으로 인천시 데이터는 결측치와 이상치를 포함하고 있다. 따라서 대부분의 인천시 데이터에서 결측치의 개수는 0보다 크고, 최대값은 매우 큰 값을 가진다.

The Incheon dataset generally contains missing values and outliers. As a result, in most cases, the missing values count is greater than 0, and the max value is unusually large.

In [ ]:
print("Description of the Incheon dataset")
print(df_imc.reset_index(drop=True).melt()["value"].describe().apply(lambda x: format(x, '.4f')))
print("Missing values count:", df_imc.isnull().sum().sum())

### Plots by Time

#### By Time

In [ ]:
def plot_by_time(df: pd.DataFrame, title: str):
    groups = df.groupby([df.index.hour, df.index.minute, df.index.second])
    mean_df = groups.apply(lambda x: x.stack().mean())

    mean_df.index = [
        datetime(year=1970, month=1, day=1, hour=h, minute=m, second=s)
        for h, m, s in mean_df.index
    ]

    plt.figure(figsize=(10, 6))
    sns.lineplot(x=mean_df.index, y=mean_df.values)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))

    plt.xlabel("Time")
    plt.ylabel("Average Value")
    plt.title(title)

    plt.grid(True)

    plt.show()

In [ ]:
plot_by_time(df_la, "METR-LA")
plot_by_time(df_pems, "PEMS-BAY")
plot_by_time(df_imc, "Incheon")

In [ ]:
def plot_by_days(df: pd.DataFrame, name: str):
    data_list = []

    for day in range(7):
        day_data = df[df.index.dayofweek == day]
        numeric_cols = day_data.select_dtypes(include=[np.number]).columns
        day_numeric = day_data[numeric_cols]
        values = day_numeric.values.flatten()
        values = values[~np.isnan(values)]
        df_day = pd.DataFrame({"Speed": values, "Days": day})
        data_list.append(df_day)

    df_all = pd.concat(data_list, ignore_index=True)

    # 요일 레이블 매핑
    day_labels = {0: "Mon", 1: "Tue", 2: "Wed", 3: "Thu", 4: "Fri", 5: "Sat", 6: "Sun"}
    df_all["Days"] = df_all["Days"].map(day_labels)
    df_all["Days"] = pd.Categorical(
        df_all["Days"],
        categories=["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"],
        ordered=True,
    )

    plt.figure(figsize=(10, 6))
    sns.boxplot(
        x="Days",
        y="Speed",
        data=df_all,
        order=["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"],
    )
    plt.xlabel("Days")
    plt.ylabel("Speed")
    plt.title(f"Data by Day in {name}")
    plt.grid(True)

    ax = plt.gca()
    formatter = FuncFormatter(lambda x, pos: f'{int(x):,}')
    ax.yaxis.set_major_formatter(formatter)
    
    plt.show()

In [ ]:
plot_by_days(df_la, "METR-LA")
plot_by_days(df_pems, "PEMS-BAY")
plot_by_days(df_imc, "Incheon")
# Lots of outliers in Incheon 

### Histogram

In [ ]:
def plot_hist(
    df: pd.DataFrame,
    name: str,
    exclude_zero: bool = False,
    y_max: float = None,
    x_max: float = None,
    bins: int = 50,
):
    values = df.values.flatten()
    values = values[~np.isnan(values)]

    if exclude_zero:
        values = values[values != 0]

    if x_max is not None:
        bin_edges = np.histogram_bin_edges(
            values, bins=bins, range=(values.min(), x_max)
        )
        bin_edges = np.append(bin_edges, np.inf)
    else:
        bin_edges = bins

    plt.figure(figsize=(10, 6))
    sns.histplot(values, bins=bin_edges, kde=True)
    plt.xlabel("Value")
    plt.title(f"Histogram of {name}")
    plt.grid(True)

    if y_max is not None:
        plt.ylim(top=y_max)

    ax = plt.gca()
    formatter = FuncFormatter(lambda x, _: f"{int(x):,}")
    ax.yaxis.set_major_formatter(formatter)
    ax.xaxis.set_major_formatter(formatter)

    plt.show()

In [ ]:
plot_hist(df_la, "METR-LA")

In [ ]:
plot_hist(df_pems, "PEMS-BAY")

인천시 데이터의 경우 일반적인 한 방향의 도로에서 8000이상의 교통량은 불가능하다. 이에 따라 8000까지만 제한하여 시각화

In [ ]:
plot_hist(df_imc, "Incheon", y_max=2000000, x_max=2000)
plot_hist(df_imc, "Incheon", y_max=100000, x_max=8000)

## Inspection of Time-Series Data Structure and Samples

### Indexes

In [ ]:
print("-- METR-LA\n", f"dtype: {df_la.index.dtype}\n", f"length: {len(df_la.index)}", end="\n\n")
print("-- PEMS-BAY\n", f"dtype: {df_pems.index.dtype}\n", f"length: {len(df_pems.index)}", end="\n\n")
print("-- Incheon\n", f"dtype: {df_imc.index.dtype}\n", f"length: {len(df_imc.index)}")

In [ ]:
print("METR-LA")
print("Is time series index:", isinstance(df_la.index, pd.DatetimeIndex))
print("Monotonic Increase:", df_la.index.is_monotonic_increasing)
print("Time Freq:", pd.infer_freq(df_la.index))
print()

print("PEMS-BAY")
print("Is time series index:", isinstance(df_pems.index, pd.DatetimeIndex))
print("Monotonic Increase:", df_pems.index.is_monotonic_increasing)
print("Time Freq:", pd.infer_freq(df_pems.index))
print()

print("Incheon")
print("Is time series index:", isinstance(df_imc.index, pd.DatetimeIndex))
print("Monotonic Increase:", df_imc.index.is_monotonic_increasing)
print("Time Freq:", pd.infer_freq(df_imc.index))
print()

In [ ]:
print("METR-LA")
time_diffs = df_la.index.to_series().diff()
time_diffs = time_diffs[1:]
gaps = time_diffs[time_diffs != pd.Timedelta(minutes=5)]

if gaps.empty:
    print("Index is 5-minutes interval")
else:
    print("Index is not 5-minutes interval")
    for idx in gaps.index:
        print(f"From: {df_la.index[df_la.index.get_loc(idx) - 1]}, To: {idx}")

print("\r\nPEMS-BAY")
time_diffs = df_pems.index.to_series().diff()
time_diffs = time_diffs[1:]
gaps = time_diffs[time_diffs != pd.Timedelta(minutes=5)]

if gaps.empty:
    print("Index is 5-minutes interval")
else:
    print("Index is not 5-minutes interval")
    for idx in gaps.index:
        print(f"From: {df_pems.index[df_pems.index.get_loc(idx) - 1]}, To: {idx}")

print("\r\nIncheon")
time_diffs = df_imc.index.to_series().diff()
time_diffs = time_diffs[1:]
gaps = time_diffs[time_diffs != pd.Timedelta(hours=1)]

if gaps.empty:
    print("Index is 1-hour interval")
else:
    print("Index is not 1-hour interval")
    for idx in gaps.index:
        print(f"From: {df_imc.index[df_imc.index.get_loc(idx) - 1]}, To: {idx}")

### Columns

In [ ]:
print("-- METR-LA\n", f"dtype: {df_la.dtypes.unique().tolist()}\n", f"length: {len(df_la.columns)}", end="\n\n")
print("-- PEMS-BAY\n", f"dtype: {df_pems.dtypes.unique().tolist()}\n", f"length: {len(df_pems.columns)}", end="\n\n")
print("-- Incheon\n", f"dtype: {df_la.dtypes.unique().tolist()}\n", f"length: {len(df_imc.columns)}")

## Outliers in Incheon Data

앞서 살펴본 바에 따르면, 인천 데이터에는 다른 데이터에서는 볼 수 없는 Outlier가 많이 존재하며 비정상적인 값들이 많다. 이를 처리할 수 있는 방법이 필요하다.

In [ ]:
idx = 0

In [ ]:
def get_hourly_data(df: pd.DataFrame, col_idx: int):
    series = df.iloc[:, col_idx]
    groups = series.groupby(
        [
            series.index.hour,
            series.index.minute,
            series.index.second,
        ]
    )
    data = groups.apply(lambda x: x.reset_index(drop=True)).reset_index()

    data["Time"] = data["level_0"]
    col_name = df.columns[col_idx]
    data = data[["Time", col_name]]

    # 열 이름 변경
    data.columns = ["Time", "Value"]

    return data, col_name

In [ ]:
def plot_iqr_outliers(df: pd.DataFrame, col_idx: int = 0):
    data, col_name = get_hourly_data(df, col_idx)

    plt.figure(figsize=(12, 6))
    sns.boxplot(x="Time", y="Value", data=data)
    plt.title(f"Outliers in {col_name}")
    plt.show()

    return col_name

In [ ]:
def plot_zscore_outliers(df: pd.DataFrame, col_idx: int = 0, z_threshold: float = 3.5):
    data, col_name = get_hourly_data(df, col_idx)
    
    error_band = []
    for hour in range(24):
        series = data[data["Time"] == hour]["Value"]
        mean = series.mean()
        std = series.std()
        
        lower_bound = mean - z_threshold * std
        upper_bound = mean + z_threshold * std
        error_band.append({"Time": hour, "Mean": mean, "Lower": lower_bound, "Upper": upper_bound})
    
    error_band = pd.DataFrame(error_band)

    outlier_data = []
    for hour in range(24):
        series = data[data["Time"] == hour]["Value"]
        z_scores = stats.zscore(series, nan_policy="omit")
        outliers = series[np.abs(z_scores) > z_threshold]
        for value in outliers:
            outlier_data.append({"Time": hour, "Value": value})
    
    outlier_data = pd.DataFrame(outlier_data)

    plt.figure(figsize=(12, 6))
    
    sns.lineplot(x="Time", y="Value", data=data, label="Average Value")
    plt.fill_between(
        error_band["Time"],
        error_band["Lower"],
        error_band["Upper"],
        color="gray",
        alpha=0.3,
        label=f"Z-Score Range ±{z_threshold}"
    )
    
    if not outlier_data.empty:
        sns.scatterplot(x="Time", y="Value", data=outlier_data, color="red", label="Outliers")
    
    plt.xlabel("Time")
    plt.ylabel(col_name)
    plt.title(f"Z-Score ±{z_threshold} Range for {col_name}")
    plt.legend()
    plt.grid(True)
    plt.show()

    return outlier_data

In [ ]:
z_score_threshold = 5

In [ ]:
idx += 1

In [ ]:
col_name = plot_iqr_outliers(df_imc, idx)
plot_zscore_outliers(df_imc, idx, z_threshold=z_score_threshold)


### Outlier 처리

이상적인 차선 하나 당 이론적 최대 통행량은 다음과 같다.

1. 제한속도 100km/h: 2200대
3. 제한속도 80km/h: 2000대
5. 제한속도 60km/h: 1800대

즉, 10km/h 줄어들때마다 100대씩 줄어든다. 이를 기준으로 다시 분석한다.

참조:
오영태, 김수희, 정성환, & 함태식. (2008). 국내ㆍ외 고속도로 용량산정방법 비교ㆍ분석 연구. 한국 ITS 학회 학술대회, 29-36.

#### 제한속도 별 도로 분류 및 통행량 분석

In [ ]:
max_speed_map = imc_metadata.set_index("LINK_ID")["MAX_SPD"].to_dict()

grouped_columns: Dict[int, List[str]] = {}
for col in df_imc.columns:
    if col in max_speed_map:
        max_speed = max_speed_map[col]
        if max_speed not in grouped_columns:
            grouped_columns[max_speed] = []
        grouped_columns[max_speed].append(col)

grouped_df_imc = {speed: df_imc[cols] for speed, cols in grouped_columns.items()}

In [ ]:
speed_limit_list = sorted(grouped_df_imc.keys())
speed_limit_list

In [ ]:
def get_ideal_road_max_flow(speed: int, lanes_count: int = 1, rate: float = 1):
    return (2200 - 10 * (100 - speed)) * lanes_count * rate

In [ ]:
y_maxes = {100: 25000, 80: 10000, 70: 50000, 60: 600000, 50: 1000000, 40: 400000, 30: 200000}
for speed in speed_limit_list:
    x_max = get_ideal_road_max_flow(speed)
    plot_hist(grouped_df_imc[speed], f"Incheon Speed {speed}", y_max=y_maxes[speed], x_max=x_max)

In [ ]:
for speed in speed_limit_list:
    plot_by_time(grouped_df_imc[speed], f"Incheon Data at Max Speed {speed}")

#### 제한속도 별 이상치 있는 센서 시각화

명백한 이상치가 있는 센서의 데이터를 시각화하고 어떤 문제가 있는지 분석

In [ ]:
max_speed_map = imc_metadata.set_index("LINK_ID")["MAX_SPD"].to_dict()
lane_map = imc_metadata.set_index("LINK_ID")["LANES"].to_dict()

In [ ]:
z_score_threshold = 5
# [30, 40, 50, 60, 70, 80, 100]
# target_max_speed = 50
idx = 0
rate = 1

# ----- #

prev_idx = idx
prev_sensor = "None"

# 최대속도로 최대 Lane 당 허용용량 설정
# Lane 수로 도로의 허용용량 설정
# 기준을 이 두개로 설정해서 다시 시각화

In [ ]:
df = df_imc
threshold = z_score_threshold


def plot_sensor_data(
    target_sensor_data: pd.Series,
    expected_road_max: float,
    title: str,
):
    plt.figure(figsize=(12, 6))

    # 전체 데이터 시각화
    sns.lineplot(x=target_sensor_data.index, y=target_sensor_data.values)
    plt.axhline(
        y=expected_road_max,
        color="red",
        linestyle="--",
        label=f"Ideal Road Max: {expected_road_max}",
    )

    plt.xlabel("Time")
    plt.ylabel("Average Value")
    plt.title(title)

    plt.show()

    # z-score 이상치 시각화
    outliers = plot_zscore_outliers(df, idx, z_threshold=threshold)
    print("Missings:", target_sensor_data.isna().sum())
    print("Outliers:", len(outliers))

In [ ]:
while True:
    if idx >= df_imc.shape[1]:
        print("End of the sensors")
        break

    target_sensor_data = df_imc.iloc[:, idx]
    values = target_sensor_data.values.flatten()
    values = values[~np.isnan(values)]

    max_speed = max_speed_map[target_sensor_data.name]
    lanes_count = lane_map[target_sensor_data.name]
    expected_road_max = get_ideal_road_max_flow(max_speed, lanes_count, rate=rate)

    exceed_count = (values > expected_road_max).sum()
    if exceed_count > 0:
        print(
            f"Previous Sensor({prev_idx}: {prev_sensor}) => Current Sensor({idx}: {target_sensor_data.name})"
        )
        print()
        non_missing_rate = (len(values) / len(target_sensor_data)) * 100
        print(
            f"Data Counts: {len(values)} / {len(target_sensor_data)} ({non_missing_rate:.2f}%, Missing Rate: {100 - non_missing_rate:.2f}%)"
        )
        print(f"Max Speed: {max_speed} km/h")
        print(f"Lanes: {lanes_count}")
        print(
            f"Ideal Flow Max: {get_ideal_road_max_flow(max_speed)} * {lanes_count} * {rate} = {expected_road_max}"
        )
        print(f"Exceed Count: {exceed_count}")

        sensor_map = plot_sensor_data(
            target_sensor_data,
            expected_road_max,
            f"Incheon Sensor {target_sensor_data.name}",
        )

        prev_idx = idx
        prev_sensor = target_sensor_data.name
        idx += 1
        break

    idx += 1

# 도로 위치 시각화
target_sensor_position: gpd.GeoDataFrame = roads_imc[
    roads_imc["LINK_ID"] == target_sensor_data.name
]
target_sensor_position.explore(
    color="blue",
    style_kwds={"style_function": lambda _: {"radius": 10}},
)

In [ ]:
month_names_korean = [
    "1월",
    "2월",
    "3월",
    "4월",
    "5월",
    "6월",
    "7월",
    "8월",
    "9월",
    "10월",
    "11월",
    "12월",
]
months = sorted(target_sensor_data.index.month.unique())

for month in months:
    month_data = target_sensor_data[target_sensor_data.index.month == month]
    month_name = month_names_korean[month - 1]

    plt.figure(figsize=(12, 6))
    sns.lineplot(x=month_data.index, y=month_data.values)
    plt.axhline(
        y=expected_road_max,
        color="red",
        linestyle="--",
        label=f"Ideal Road Max: {expected_road_max}",
    )

    plt.xlabel("시간")
    plt.title(
        f"{target_sensor_data.name} - {month_name} Max(Speed: {max_speed} km/h, Flow: <{expected_road_max})"
    )

    plt.grid(True)
    plt.show()

Outlier 기준은 다음과 같다.

1. 10000 이상의 값은 비정상으로 인식
2. z-score 5 이상은 비정상으로 인식

In [ ]:
from metr.components.metr_imc.outlier import AbsoluteOutlierProcessor, HourlyInSensorOutlierProcessor

outlier_proc_1 = AbsoluteOutlierProcessor(threshold=10000)
outlier_proc_2 = HourlyInSensorOutlierProcessor(threshold=z_score_threshold)

In [ ]:
metr_imc.reset_data()
metr_imc.remove_outliers([outlier_proc_1, outlier_proc_2])
df_imc_outliers_removed = metr_imc.data

In [ ]:
plot_iqr_outliers(df_imc_outliers_removed, idx)
plot_zscore_outliers(df_imc_outliers_removed, idx, z_threshold=z_score_threshold)

In [ ]:
sensor_name = col_name
print(df_imc[sensor_name][df_imc[sensor_name] > 10000])
print("IMC:", df_imc[sensor_name].count())
print("IMC_NoOut:", df_imc_outliers_removed[sensor_name].count())


## Missing Data in Incheon Data

In [ ]:
def count_columns_by_missing_range(df: pd.DataFrame):
    total_cols = len(df.columns)
    no_missing = len(df.columns[df.isnull().sum() == 0])
    missing_1_to_100 = len(df.columns[(df.isnull().sum() > 0) & (df.isnull().sum() <= 100)])
    missing_101_to_500 = len(df.columns[(df.isnull().sum() > 100) & (df.isnull().sum() <= 500)])
    missing_501_to_750 = len(df.columns[(df.isnull().sum() > 500) & (df.isnull().sum() <= 750)])
    missing_751_above = len(df.columns[df.isnull().sum() > 750])

    print(f"   Total Columns: {total_cols}")
    print(f"   No Missing: {no_missing}")
    print(f"   1 to 100 Missing: {missing_1_to_100}")
    print(f"   101 to 500 Missing: {missing_101_to_500}")
    print(f"   501 to 750 Missing: {missing_501_to_750}")
    print(f"   751 or more Missing: {missing_751_above}")

# METR-LA
print("-- METR-LA")
count_columns_by_missing_range(df_la)
print()

# PEMS-BAY
print("-- PEMS-BAY")
count_columns_by_missing_range(df_pems)
print()

# Incheon
print("-- Incheon")
count_columns_by_missing_range(df_imc)
print()

#### Visualization of Missing Data

In [ ]:
def visualize_missingno(df: pd.DataFrame):
    msno.matrix(df)
    plt.show()

    msno.heatmap(df, labels=False)
    plt.show()

    msno.bar(df)
    plt.show()

In [ ]:
df_temp = df_imc_outliers_removed[df_imc_outliers_removed.isnull().sum(axis=1) > 500]
df_temp.shape

In [ ]:
df_temp

In [ ]:
visualize_missingno(df_temp)

------------------------------------------------------------------------------------------------

missingno는 결측값 시각화 패키지이며 각 함수는 다음의 의미와 같다.
1. matrix: 빈 공간은 결측값을 나타내며 이를 통해 결측 패턴을 파악.
2. heatmap: 열 간 결측값 상관 계수를 히트맵으로 표시. 변수 간 결측값의 연관성 파악.
3. bar: 결측값 갯수를 막대로 표시. 각 열(변수)의 결측값 갯수 파악.

In [ ]:
sample_rate = 0.1

data_columns = df_imc_sorted.columns
num_columns_to_select = int(len(data_columns) * sample_rate)
selected_columns = np.random.choice(data_columns, num_columns_to_select, replace=False)
sampled_df = df_imc_sorted[selected_columns]

print(f"샘플링된 데이터 프레임의 크기: {sampled_df.shape}")
visualize_missingno(sampled_df)

Matrix에서 특정 날짜에서 결측값이 많아지는 경향을 볼 수 있다. 특히 데이터 시작 부분의 일정 기간 동안 결측값이 많이 관찰되었다. 특정 날짜에서 대부분의 센서들이 동작하지 않았을 가능성이 높다. 또한, 데이터 시작 부분의 데이터는 제외하는 것을 고려할 수 있다.

Heatmap 분석에서는 결측값의 상관관계가 높은 경우가 많이 확인되었다. 일반적으로 이 경우 결측값 형태가 MAR(Missing At Random)이라고 판단한다. 다만 각 열이 모두 독립적인 센서임을 감안할 때, 각 센서가 영향을 주었을 가능성은 낮고 중앙 시스템에서 문제가 됬을 가능성이 높기 때문에 이 점을 감안할 필요가 있다.

Bar 분석에서는 대부분 결측치가 많다는 것을 확인할 수 있다.

In [ ]:
sample_traffic_data = df_imc[df_imc.columns[df_imc.isnull().sum() == 0][0]]

plt.figure(figsize=(12, 2))
sns.lineplot(sample_traffic_data)
plt.title(f"{sample_traffic_data.name} Road Traffic Data")
plt.xlabel("Date Time")
plt.show()

최종적으로는 결측값을 포함한 전체 데이터셋을 기반으로 모델을 생성하고 성능을 평가해야 하지만 결측값 문제는 현재 집중하고 있는 연구 방향과는 독립적인 문제로 판단된다. 또한, 모든 데이터를 포함하면 데이터의 크기가 커져 학습 시간이 길어지는 문제도 있다. 따라서 현재는 데이터를 줄여 효율성을 높이고자 먼저 결측값이 없는 데이터를 사용하여 모델을 생성하고, 이 모델이 유효한지 판단한 후, 결측값 문제를 추가로 해결하는 방향으로 연구를 진행하고자 한다.

In [ ]:
no_missing_columns = df_imc.columns[df_imc.isnull().sum() == 0].to_list()
less_500_missing_columns = df_imc.columns[df_imc.isnull().sum() < 500].to_list()
less_750_missing_columns = df_imc.columns[df_imc.isnull().sum() < 750].to_list()

In [ ]:
import os
from songdo_metr.dataset.metr_imc.converter.graph_sensor_locations import SensorView

if not os.path.exists("../datasets/metr-imc/miscellaneous/no_missing.shp"):
    view = SensorView("../datasets/metr-imc/graph_sensor_locations.csv")
    view.set_filter(no_missing_columns)
    view.export_to_file("../datasets/metr-imc/miscellaneous", "no_missing.shp")
    view.set_filter(less_500_missing_columns)
    view.export_to_file("../datasets/metr-imc/miscellaneous", "missing_500.shp")
    view.set_filter(less_750_missing_columns)
    view.export_to_file("../datasets/metr-imc/miscellaneous", "missing_750.shp")

결측치가 없는 데이터를 사용하는 것이 가장 이상적이지만, 이 경우 노드의 수가 너무 줄어들어 공간 정보가 모델에 충분히 반영되지 않을 가능성이 있다. 따라서, 기존에 많이 사용하는 METR-LA, PEMS-BAY 등의 데이터셋 크기와 유사한 수준으로 조정하였다. 결측치 허용 기준을 여러 개 비교한 결과, 500 정도가 기존 데이터셋과 유사한 크기를 유지하면서도 공간 정보를 적절히 반영할 수 있을 것으로 예상되었다.

- 결측치가 하나도 없는 센서 노드

![Missing_0](../docs/Missing_0.png)

- 결측치 500개 미만의 센서 노드

![Missing_500](../docs/Missing_500.png)

- 결측치 750개 미만의 센서 노드

![Missing_750](../docs/Missing_750.png)

이에 따라 500개 미만의 결측치를 가진 센서 노드만 별도로 추출하여 데이터셋을 다시 생성하였다.

In [ ]:
from songdo_traffic_core.dataset.metr_imc.generator import MetrImcSubsetGenerator

generator = MetrImcSubsetGenerator(
    nodelink_dir="../datasets/metr-imc/nodelink",
    imcrts_dir="../datasets/metr-imc/imcrts",
    metr_imc_dir="../datasets/metr-imc/",
)

In [ ]:
l500_missing_df, l500_columns = generator.process_metr_imc(less_500_missing_columns)
l500_missing_df.info()

In [ ]:
visualize_missingno(l500_missing_df)

위 그래프의 경우 원래데이터와 다르게 결측치 분포의 편차는 많이 제거되었다. 특정 기간동안 보이던 결측치 또한 많이 제거되어 있음을 확인할 수 있다. 센서마다 보이던 결측치의 상관관계도 많이 줄어들었다. 그러나 여전히 상관관계가 있는 결측치들이 있으며 이것은 여전히 결측치가 MAR임을 알 수 있다.

In [ ]:
from sklearn.cluster import KMeans


missing_patterns = l500_missing_df.isnull().astype(int)
kmeans = KMeans(n_clusters=8, random_state=42)
clusters = kmeans.fit_predict(missing_patterns.T)
clustered_df = pd.DataFrame({'sensor': missing_patterns.columns, 'cluster': clusters})
for cluster in set(clusters):
    print(f"Cluster {cluster}: {clustered_df[clustered_df['cluster'] == cluster]['sensor'].tolist()}")

In [ ]:
visualize_clusters = True

if visualize_clusters:
    for cluster in set(clusters):
        cluster_sensors = clustered_df[clustered_df['cluster'] == cluster]['sensor'].tolist()
        print("="*10, "Cluster", cluster, "="*10)
        plt.figure(figsize=(6, 2))
        msno.matrix(l500_missing_df[cluster_sensors])
        plt.title(f'Matrix Plot for Cluster {cluster}')
        plt.show()

        plt.figure(figsize=(6, 2))
        msno.heatmap(l500_missing_df[cluster_sensors], labels=False)
        plt.title(f'Heatmap Plot for Cluster {cluster}')
        plt.show()